In [ ]:
from IPython import display

In [ ]:
try:
    import nbformat
except:
    !pip install nbformat
display.clear_output()

In [ ]:
%run /content/drive/MyDrive/house_predict/process/test_24102025/model/train.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
+ action: train_feat02
+ feat_path: /content/drive/MyDrive/house_predict/process/exps/data.npz
+ seed: 42
+ exp_dir: /content/drive/MyDrive/house_predict/process/exps
+ exp_name: trainbase_24102025
+ data_dir: /content/drive/MyDrive/house_predict/data
+ verbose: True
+ save_dir: /content/drive/MyDrive/house_predict/process/exps/trainbase_24102025
---------- information ----------
train-col: {'HouseStyle', 'RoofStyle', 'Fireplaces', 'BsmtFinSF1', 'ExterCond', 'Neighborhood', 'PoolArea', 'MiscVal', 'LandContour', 'Condition1', 'Functional', 'KitchenQual', 'OverallQual', 'MasVnrArea', 'GrLivArea', 'OpenPorchSF', 'CentralAir', 'HeatingQC', 'TotRmsAbvGrd', 'OverallCond', 'BsmtCond', '2ndFlrSF', 'Street', 'Foundation', 'MSSubClass', 'TotalBsmtSF', 'LandSlope', 'HalfBath', 'YearRemodAdd', 'Exterior1st', 'SaleType', 'GarageType', 'BldgType', 'BsmtHalfBath', 'FullBath', 'Utilities', 'GarageQual', 'BedroomAbv

# Import lib

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

data = np.load("/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025/data02.npz", allow_pickle=True)

In [ ]:
with np.load('/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025/data02.npz') as z:
    print(z.files)

['train_data', 'test_data', 'train_columns', 'test_columns', 'allow_pickle']


In [ ]:
# Lấy dữ liệu train và test
df_train = pd.DataFrame(data = data['train_data'], columns= data['train_columns'])
df_test = pd.DataFrame(data['test_data'], columns=df_train.drop(columns=['Output']).columns)

In [ ]:
df_train

,MasVnrArea,LotFrontage,GarageYrBlt,Electrical,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,MiscFeature,...,MoSold,YrSold,SaleType,SaleCondition,TotalLiving,Qual_LivArea,GarageScore,TotalAmenities,Loc_Style_Cond,Qual_Exter_Overall
0,196.0,65.0,2003.0,0.0,5.0,2.0,3.0,5.0,0.0,0.0,...,2.0,2008.0,8.0,4.0,2566.0,17962.0,550.0,3.0,26.0,17.0
1,0.0,80.0,1976.0,0.0,5.0,2.0,3.0,5.0,0.0,0.0,...,5.0,2007.0,8.0,4.0,2524.0,15144.0,462.0,4.0,185.0,24.0
2,162.0,68.0,2001.0,0.0,5.0,2.0,3.0,5.0,0.0,0.0,...,9.0,2008.0,8.0,4.0,2706.0,18942.0,610.0,4.0,26.0,17.0
3,0.0,60.0,1998.0,0.0,1.0,1.0,3.0,5.0,0.0,0.0,...,2.0,2006.0,8.0,0.0,2473.0,17311.0,645.0,3.0,39.0,25.0
4,350.0,84.0,2000.0,0.0,5.0,2.0,3.0,5.0,0.0,0.0,...,12.0,2008.0,8.0,4.0,3343.0,26744.0,839.0,4.0,115.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.0,62.0,1999.0,0.0,5.0,2.0,3.0,5.0,0.0,0.0,...,8.0,2007.0,8.0,4.0,2600.0,15600.0,462.0,4.0,63.0,24.0
1456,119.0,85.0,1978.0,0.0,5.0,1.0,3.0,5.0,0.0,0.0,...,2.0,2010.0,8.0,4.0,3615.0,21690.0,502.0,5.0,107.0,24.0
1457,0.0,66.0,1941.0,0.0,5.0,2.0,3.0,5.0,0.0,2.0,...,5.0,2010.0,8.0,4.0,3492.0,24444.0,253.0,5.0,41.0,2.0
1458,0.0,68.0,1950.0,1.0,5.0,1.0,3.0,5.0,0.0,0.0,...,4.0,2010.0,8.0,4.0,2156.0,10780.0,241.0,2.0,94.0,23.0


In [ ]:
df_test

,MasVnrArea,LotFrontage,GarageYrBlt,Electrical,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,MiscFeature,...,MoSold,YrSold,SaleType,SaleCondition,TotalLiving,Qual_LivArea,GarageScore,TotalAmenities,Loc_Style_Cond,Qual_Exter_Overall
0,0.0,80.0,1961.0,0.0,5.0,1.0,3.0,5.0,0.0,0.0,...,6.0,2010.0,9.0,4.0,1778.0,8890.0,731.0,2.0,98.0,20.0
1,108.0,81.0,1958.0,0.0,5.0,1.0,3.0,5.0,0.0,1.0,...,6.0,2010.0,9.0,4.0,2658.0,15948.0,313.0,2.0,98.0,21.0
2,0.0,74.0,1997.0,0.0,5.0,3.0,3.0,5.0,0.0,0.0,...,3.0,2010.0,9.0,4.0,2557.0,12785.0,484.0,4.0,61.0,20.0
3,20.0,78.0,1998.0,0.0,5.0,3.0,3.0,5.0,0.0,0.0,...,6.0,2010.0,9.0,4.0,2530.0,15180.0,472.0,4.0,61.0,21.0
4,0.0,43.0,1992.0,0.0,5.0,2.0,3.0,5.0,0.0,0.0,...,1.0,2010.0,9.0,4.0,2560.0,20480.0,508.0,3.0,172.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.0,21.0,1979.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,2006.0,9.0,4.0,1638.0,6552.0,0.0,2.0,82.0,19.0
1455,0.0,21.0,1970.0,0.0,3.0,1.0,3.0,5.0,0.0,0.0,...,4.0,2006.0,9.0,0.0,1638.0,6552.0,287.0,2.0,81.0,19.0
1456,0.0,160.0,1960.0,0.0,1.0,1.0,3.0,5.0,0.0,0.0,...,9.0,2006.0,9.0,0.0,2448.0,12240.0,578.0,3.0,86.0,20.0
1457,0.0,62.0,1979.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,7.0,2006.0,9.0,4.0,1882.0,9410.0,0.0,2.0,91.0,20.0


In [ ]:
df_test.isnull().sum()[df_test.isnull().sum() > 0]

,0


In [ ]:
df_train.isnull().sum()[df_train.isnull().sum() > 0]

,0


In [ ]:
X_train = df_train.drop(columns=['Output'], axis=1)
y_train = df_train['Output']

In [ ]:
X_test = df_test

# LinearRegression




In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Tạo pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge())  # hoặc Lasso(), bạn có thể thay thử
])

# Thiết lập hyperparameters
param_grid = {
    'regressor': [Ridge(), Lasso()],
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# Sử dụng K-Fold cho regression (không Stratified)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search với nhiều scoring metrics
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring={'r2': 'r2', 'rmse': 'neg_root_mean_squared_error'},
    refit='r2',  # Chọn model có R² cao nhất
    n_jobs=-1,
    verbose=1
)

# Train model
print("\n Bắt đầu training với GridSearch...")
grid_search.fit(X_train, y_train)

# Kết quả
print("\n Best parameters:", grid_search.best_params_)
print("🏆 Best cross-validation R²: {:.4f}".format(grid_search.best_score_))

# Dự đoán trên train
y_pred_train = grid_search.predict(X_train)
rmse = mean_squared_error(y_train, y_pred_train) ** 0.5
r2 = r2_score(y_train, y_pred_train)
print(f"\n📊 Train RMSE: {rmse:.4f}")
print(f"📈 Train R²: {r2:.4f}")

# Hiển thị tất cả kết quả
cv_results = grid_search.cv_results_
print("\n Cross-validation results:")
for i in range(len(cv_results['params'])):
    print(f"Params: {cv_results['params'][i]}")
    print(f"  R²: {cv_results['mean_test_r2'][i]:.4f} (+/- {cv_results['std_test_r2'][i]:.4f})")
    print(f"  RMSE: {-cv_results['mean_test_rmse'][i]:.4f} (+/- {cv_results['std_test_rmse'][i]:.4f})")



 Bắt đầu training với GridSearch...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

 Best parameters: {'regressor': Ridge(), 'regressor__alpha': 100}
🏆 Best cross-validation R²: 0.7669

📊 Train RMSE: 29278.4447
📈 Train R²: 0.8641

 Cross-validation results:
Params: {'regressor': Ridge(), 'regressor__alpha': 0.001}
  R²: 0.6679 (+/- 0.3859)
  RMSE: 39575.4559 (+/- 19521.6961)
Params: {'regressor': Ridge(), 'regressor__alpha': 0.01}
  R²: 0.6680 (+/- 0.3857)
  RMSE: 39571.6361 (+/- 19515.1387)
Params: {'regressor': Ridge(), 'regressor__alpha': 0.1}
  R²: 0.6690 (+/- 0.3838)
  RMSE: 39533.9220 (+/- 19450.2240)
Params: {'regressor': Ridge(), 'regressor__alpha': 1}
  R²: 0.6779 (+/- 0.3669)
  RMSE: 39197.6917 (+/- 18859.5162)
Params: {'regressor': Ridge(), 'regressor__alpha': 10}
  R²: 0.7208 (+/- 0.2845)
  RMSE: 37507.8178 (+/- 15756.7213)
Params: {'regressor': Ridge(), 'regressor__alpha': 100}
  R²: 0.7669 (+/- 0.1985)
  RMSE: 35389.4272 (+/- 12215.5786)
Params: {'regressor

In [ ]:
# Model tốt nhất
best_model_lr = grid_search.best_estimator_

# Dự đoán trên tập train
y_pred_train = best_model_lr.predict(X_train)

# Đánh giá mô hình
train_r2 = r2_score(y_train, y_pred_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
train_rmse = train_mse ** 0.5

print("\n=== KẾT QUẢ TRÊN TRAINING SET ===")
print(f"R² Score      : {train_r2:.4f}")
print(f"MAE           : {train_mae:.4f}")
print(f"MSE           : {train_mse:.4f}")
print(f"RMSE          : {train_rmse:.4f}")


=== KẾT QUẢ TRÊN TRAINING SET ===
R² Score      : 0.8641
MAE           : 17910.6996
MSE           : 857227322.8071
RMSE          : 29278.4447


# Dump Linear Regression

In [ ]:
import joblib

# Giả sử model đã train xong
# ví dụ: best_model_lr.fit(X_train, y_train)

# Lưu riêng model
joblib.dump(best_model_lr, '/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025/linear_regression_model.pkl')
print("✅ Model đã được lưu vào linear_regression_model.pkl")

# Lưu thêm thông tin đi kèm (đề phòng sau này dùng lại)
package = {
    'model': best_model_lr,
    'scaler': scaler if 'scaler' in locals() else None,
    'feature_names': X_train.columns.tolist(),
    'training_date': '2025-10-26'
}

joblib.dump(package, '/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025/linear_regression_package.pkl')
print("✅ Package (model + scaler + features) đã được lưu")


✅ Model đã được lưu vào linear_regression_model.pkl
✅ Package (model + scaler + features) đã được lưu


In [ ]:
# 🔹 Load model đã lưu
model_package = joblib.load('/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025/linear_regression_package.pkl')
model = model_package['model']

# 🔹 Nếu có scaler thì load ra
scaler = model_package.get('scaler', None)

# 🔹 Đọc lại file test gốc để lấy cột Id
df_test_raw = pd.read_csv('/content/drive/MyDrive/house_predict/data/test.csv')
ids = df_test_raw['Id']  # Lưu cột Id gốc (từ 1461–2919)

# 🔹 Scale dữ liệu nếu có scaler
if scaler is not None:
    X_test = scaler.transform(df_test)
else:
    X_test = df_test.values

# 🔹 Dự đoán
y_pred_test = model.predict(X_test)

# 🔹 Tạo file submission
submission = pd.DataFrame({
    'Id': ids,
    'SalePrice': y_pred_test
})

# 🔹 Lưu file CSV
submission.to_csv('/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025/submission.csv', index=False)
print("💾 File submission.csv đã được tạo thành công!")

💾 File submission.csv đã được tạo thành công!
